### 당신은 해외로 사업을 확장하고자 하는 기업에서 Data Engineer로 일하고 있습니다. 경영진에서 GDP가 높은 국가들을 대상으로 사업성을 평가하려고 합니다.

* 이 자료는 앞으로 경영진에서 지속적으로 요구할 것으로 생각되기 때문에 자동화된 스크립트를 만들어야 합니다.

1. IMF에서 제공하는 국가별 GDP를 구하세요. (https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29)



In [19]:
import sys
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL을 설정합니다.
url = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)'

# requests를 사용하여 웹 페이지를 가져옵니다.
response = requests.get(url)

# 웹 페이지의 내용을 파싱합니다.
soup = BeautifulSoup(response.content, 'html.parser')


In [28]:
table = soup.find('table', {'class': 'wikitable sortable sticky-header-multi static-row-numbers'})
rows = table.find_all('tr')
data = []

for row in rows[2:]:  # 첫 두 행은 헤더이므로 건너뜀
    cols = row.find_all('td')
    if len(cols) < 3:
        continue
    country = cols[0].text.strip()
    gdp = cols[1].text.strip().replace(',', '')
    # region 정보는 테이블에 없으므로 빈 문자열로 초기화
    region = ''
    
    try:
        gdp = float(gdp)
    except ValueError:
        continue

    data.append([country, gdp, region])

# 데이터프레임 생성
df = pd.DataFrame(data, columns=['Country', 'GDP (Millions of USD)', 'Region'])
df

,Country,GDP (Millions of USD),Region
0,World,109529216.0,
1,United States,28781083.0,
2,China,18532633.0,
3,Germany,4591100.0,
4,Japan,4110452.0,
...,...,...,...
190,Kiribati,311.0,
191,Palau,308.0,
192,Marshall Islands,305.0,
193,Nauru,161.0,


2. 국가별 GDP를 확인할 수 있는 테이블을 만드세요.

3. 해당 테이블에는 GDP가 높은 국가들이 먼저 나와야 합니다.

4. GDP의 단위는 1B USD이어야 하고 소수점 2자리까지만 표시해 주세요.



5. IMF에서 매년 2회 이 자료를 제공하기 때문에 정보가 갱신되더라도 해당 코드를 재사용해서 정보를 얻을 수 있어야 합니다.

6. GDP가 100B USD이상이 되는 국가만을 구해서 화면에 출력해야 합니다.



7. 각 Region별로 top5 국가의 GDP 평균을 구해서 화면에 출력해야 합니다. 

In [29]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_gdp_data():
    # 위키피디아에서 GDP 데이터를 스크랩
    url = 'https://en.wikipedia.org/wiki/List_of_countries_by_GDP_(nominal)'
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # IMF 테이블 찾기
    table = soup.find('table', {'class': 'wikitable sortable sticky-header-multi static-row-numbers'})

    # 테이블 파싱
    rows = table.find_all('tr')
    data = []

    for row in rows[2:]:  # 첫 두 행은 헤더이므로 건너뜀
        cols = row.find_all('td')
        if len(cols) < 3:
            continue
        country = cols[0].text.strip()
        gdp = cols[1].text.strip().replace(',', '')
        # region 정보는 테이블에 없으므로 빈 문자열로 초기화
        region = ''
        
        try:
            gdp = float(gdp)
        except ValueError:
            continue

        data.append([country, gdp, region])

    # 데이터프레임 생성
    df = pd.DataFrame(data, columns=['Country', 'GDP (Millions of USD)', 'Region'])

    # GDP를 1B USD로 변환 및 소수점 2자리까지 표시
    df['GDP (Billions of USD)'] = df['GDP (Millions of USD)'] / 1000
    df = df[['Country', 'GDP (Billions of USD)', 'Region']]
    df['GDP (Billions of USD)'] = df['GDP (Billions of USD)'].apply(lambda x: round(x, 2))

    # GDP가 높은 순서대로 정렬
    df_sorted = df.sort_values(by='GDP (Billions of USD)', ascending=False)
    
    return df_sorted

def print_high_gdp_countries(df):
    # GDP가 100B USD 이상인 국가 필터링
    df_high_gdp = df[df['GDP (Billions of USD)'] >= 100]

    # 결과 출력
    print("GDP가 100B USD 이상인 국가 목록:")
    print(df_high_gdp.to_string(index=False))

def print_region_top5_avg_gdp(df):
    # Region 정보가 없으므로 임의로 데이터 생성 (테스트 목적)
    # 실제 데이터에는 지역 정보를 포함한 CSV 등을 이용해 추가 필요
    df.loc[df['Country'] == 'United States', 'Region'] = 'North America'
    df.loc[df['Country'] == 'China', 'Region'] = 'Asia'
    df.loc[df['Country'] == 'Japan', 'Region'] = 'Asia'
    df.loc[df['Country'] == 'Germany', 'Region'] = 'Europe'
    df.loc[df['Country'] == 'India', 'Region'] = 'Asia'
    # 기타 국가에 대해서도 비슷하게 Region 데이터 추가 필요

    # Region별 top 5 국가의 GDP 평균 계산
    regions = df['Region'].unique()
    region_top5_avg_gdp = []

    for region in regions:
        if region == '':
            continue
        df_region = df[df['Region'] == region]
        top5 = df_region.nlargest(5, 'GDP (Billions of USD)')
        avg_gdp = top5['GDP (Billions of USD)'].mean()
        region_top5_avg_gdp.append([region, round(avg_gdp, 2)])

    df_region_avg = pd.DataFrame(region_top5_avg_gdp, columns=['Region', 'Average GDP of Top 5 (Billions of USD)'])

    print("\nRegion별 top 5 국가의 GDP 평균:")
    print(df_region_avg.to_string(index=False))

# 메인 실행 부분
df_sorted = get_gdp_data()
print_high_gdp_countries(df_sorted)
print_region_top5_avg_gdp(df_sorted)


GDP가 100B USD 이상인 국가 목록:
             Country  GDP (Billions of USD) Region
               World              109529.22       
       United States               28781.08       
               China               18532.63       
             Germany                4591.10       
               Japan                4110.45       
               India                3937.01       
      United Kingdom                3495.26       
              France                3130.01       
              Brazil                2331.39       
               Italy                2328.03       
              Canada                2242.18       
              Russia                2056.84       
              Mexico                2017.03       
           Australia                1790.35       
         South Korea                1760.95       
               Spain                1647.11       
           Indonesia                1475.69       
         Netherlands                1142.51       
      